# Pregunta 2: Schnorr signature

## Imports

In [14]:
import hashlib

## Helper functions

In [15]:
# Function extracted from T2

def exp_mod(a: int, b: int, n: int) -> int:
    """
    Modular exponentation algorithm

    Args:
        a (int): number >= 0
        b (int): number >= 0
        n (int): number > 0

    Returns:
        int: a**b mod n
    """
    a = a % n
    if n == 1:
        return 0
    if b == 0:
        return 1
    if a == 0:
        return 0
    c = 1
    while b > 0:
        if ((b % 2) != 0):
            c = (c * a) % n
        b //= 2
        a = (a * a) % n
    return c


In [16]:
def read_grupo(pathname="grupo.txt"):
    """
    Loads global constants p, g, and q from file

    Arguments:
        pathname (str): path to file

    Returns:
        tuple[int]: values p, g, and q extracted from file.
    """
    with open(pathname, 'r') as grupo_file:
        grupo = grupo_file.readlines()
        p = int(grupo[0].trim(), 16)
        g = int(grupo[1].trim(), 16)
        q = int(grupo[2].trim(), 16)
    return p, g, q


In [19]:
def generar_clave_ElGamal():
    """
    Generates a public and private key with ElGamal protocol, with base values extracted from
    grupo.txt file.

    Stores the private key in private_key.txt file and the public key in public_key.txt file
    """
    p, g, q = read_grupo()
    pass

def firmar_Schnorr(m: str) -> tuple[int, int]:
    """
    Signs a message with Schnorr protocol, using the private key stored in private_key.txt file, and
    variables stored in grupo.txt file.

    Arguments:
        m (str): Message to be signed
    
    Returns:
        tuple[int, int]: message Schnorr signature in the form of (e, s)

    """
    pass

def verificar_firma_Schnorr(m: str, firma: tuple[int, int]) -> bool:
    """
    """
    pass


def md5(m: str) -> int:
    """
    Hashes a message using md5 function

    Arguments:
        m (str): Message to be hashed

    Returns:
        int: Hash of the message in the form of int
    """
    return int.from_bytes(hashlib.md5(bytes(m, 'utf-8')).digest(), 'big')


In [20]:
exp_mod(g, q, p)
md5("qweasdasdqwda")

63196945924649378879387955663894122729

## Tests

### Generate key

In [ ]:
if __name__ == "__main__":
    pass